In [16]:
# all imports
import pandas as pd
import numpy as np
import random
from random import randint
from google.colab import drive

In [17]:
# Reaload everything from drive incase the user edited any derived KPI values.
drive.mount('/content/drive', force_remount=True)
xls = pd.read_excel("/content/drive/My Drive/plan_data/constraints.xlsx",
                    sheet_name=["plan","schedule", "derived_kpis"])

# Access individual sheets using sheet names
plan = xls["plan"]
schedule = xls["schedule"]
derived_kpis = xls["derived_kpis"]

Mounted at /content/drive


In [18]:
# Derive the Actual count using efficiency
derived_kpis['ftt_rectified_sum'] = (derived_kpis['target_efficiency'] *
                                    derived_kpis['clock_mins'] *
                                    derived_kpis['Man power'])/ (derived_kpis['smv'] * 100)
derived_kpis['ftt_rectified_sum'] = derived_kpis['ftt_rectified_sum'].round()
derived_kpis['ftt'] = derived_kpis['ftt_rectified_sum'] * derived_kpis['ftt_ratio']
derived_kpis['ftt'] = derived_kpis['ftt'].round()
derived_kpis['rectified'] = derived_kpis['ftt_rectified_sum'] - derived_kpis['ftt']
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,...,rectified,direct_rejects,defective_rate_min,defective,reject,defect_increment,defect,date,module_id,TPPM
0,2024-07-26,F1M7,1,8.28,23,5,5,4,37,60,...,23.0,2,12,25,4,0,25,2024-07-26,F1M7,4.210606
1,2024-07-26,F1M7,2,8.28,23,5,5,4,37,60,...,9.0,4,4,28,23,3,31,2024-07-26,F1M7,4.210606
2,2024-07-26,F1M7,3,8.28,23,5,5,4,37,60,...,32.0,5,12,40,13,3,43,2024-07-26,F1M7,4.210606
3,2024-07-26,F1M7,4,8.28,23,5,5,4,37,60,...,31.0,0,14,32,1,0,32,2024-07-26,F1M7,4.210606
4,2024-07-26,F1M7,5,8.28,23,5,5,4,37,90,...,27.0,1,9,30,4,1,31,2024-07-26,F1M7,4.210606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,5,4,29,60,...,12.0,4,8,27,19,2,29,2024-08-01,F1M8,3.086364
150,2024-08-01,F1M8,8,8.28,14,6,5,4,29,60,...,28.0,5,14,35,12,2,37,2024-08-01,F1M8,3.086364
151,2024-08-01,F1M8,9,8.28,14,6,5,4,29,60,...,25.0,1,16,27,3,1,28,2024-08-01,F1M8,3.086364
152,2024-08-01,F1M8,10,8.28,14,6,5,4,29,60,...,23.0,0,12,25,2,3,28,2024-08-01,F1M8,3.086364


Defective rate = (Defective  x 100 )/ (FTT + Defective + Reject)

We know that Reject pcs should be greater than or equal Defective - Rectified.

**Reject (min) =  Defective - Rectified**


We also know that the Rectified should be less than or equal to Defective.

**Defective(min) = Rectified **


Defective rate = (Defective  x 100 )/ (FTT + Defective + Defective - Rectified + Delta)

Defective goes down => numerators goes down 100 times and denominator goes down 2* => defective rate goes down. Minimum defective rate is given at minimum Defectives i.e. Defective = Rectified


Defective rate (min) = (Rectified x 100) / (FTT + 2 x Rectified - Rectified +   Delta(max))


Defective rate (min) = (Rectified x 100) / (FTT + Rectified +  Delta(max))


In [19]:
# This indicates that each line may directly reject upto 6 garments that are not
# added as defectives first
max_direct_rejects = 6
derived_kpis['direct_rejects']= np.random.randint(0, max_direct_rejects, size=len(derived_kpis))

# Implementing minimum defective rate where defectives = rectified
derived_kpis['defective_rate_min'] = np.ceil((derived_kpis['rectified']*100 )/ (derived_kpis['ftt'] + derived_kpis['rectified'] + derived_kpis['direct_rejects'])).astype(int)
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,...,rectified,direct_rejects,defective_rate_min,defective,reject,defect_increment,defect,date,module_id,TPPM
0,2024-07-26,F1M7,1,8.28,23,5,5,4,37,60,...,23.0,3,12,25,4,0,25,2024-07-26,F1M7,4.210606
1,2024-07-26,F1M7,2,8.28,23,5,5,4,37,60,...,9.0,0,4,28,23,3,31,2024-07-26,F1M7,4.210606
2,2024-07-26,F1M7,3,8.28,23,5,5,4,37,60,...,32.0,2,13,40,13,3,43,2024-07-26,F1M7,4.210606
3,2024-07-26,F1M7,4,8.28,23,5,5,4,37,60,...,31.0,4,14,32,1,0,32,2024-07-26,F1M7,4.210606
4,2024-07-26,F1M7,5,8.28,23,5,5,4,37,90,...,27.0,5,9,30,4,1,31,2024-07-26,F1M7,4.210606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,5,4,29,60,...,12.0,5,8,27,19,2,29,2024-08-01,F1M8,3.086364
150,2024-08-01,F1M8,8,8.28,14,6,5,4,29,60,...,28.0,4,14,35,12,2,37,2024-08-01,F1M8,3.086364
151,2024-08-01,F1M8,9,8.28,14,6,5,4,29,60,...,25.0,2,16,27,3,1,28,2024-08-01,F1M8,3.086364
152,2024-08-01,F1M8,10,8.28,14,6,5,4,29,60,...,23.0,3,12,25,2,3,28,2024-08-01,F1M8,3.086364


**At this point, the programe will provide a symentically correct target_defective rate (min) value and ask the user to alter. You can go to the excel sheet and edit values. Afterwards the sheet will be reloaded and any remaining misfits will be set to the default minimum.**

In [20]:
# Find rows where targets are semantically unccceptable
for index, row in derived_kpis.iterrows():
    if row['target_defective_rate'] < row['defective_rate_min']:
        print(f"please change target_defective_rate value at row  {index} to a value larger than or equal to {row['defective_rate_min']}")

please change target_defective_rate value at row  59 to a value larger than or equal to 18
please change target_defective_rate value at row  63 to a value larger than or equal to 16
please change target_defective_rate value at row  85 to a value larger than or equal to 17
please change target_defective_rate value at row  99 to a value larger than or equal to 13
please change target_defective_rate value at row  143 to a value larger than or equal to 14


In [21]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='derived_kpis', index=False)



In [22]:
# Reload the excel sheet where user ay have edited target_defective-rate
drive.mount('/content/drive', force_remount=True)
xls = pd.read_excel("/content/drive/My Drive/plan_data/constraints.xlsx",
                    sheet_name=["plan","schedule", "derived_kpis"])

# Access individual sheets using sheet names
# plan = xls["plan"]
# schedule = xls["schedule"]
derived_kpis = xls["derived_kpis"]

Mounted at /content/drive


In [23]:
# Set any target_defective_rate values that are still incorrect to the default minimum value.
for index, row in derived_kpis.iterrows():
    if row['target_defective_rate'] < row['defective_rate_min']:
        derived_kpis.at[index, 'target_defective_rate'] = row['defective_rate_min']
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,...,rectified,direct_rejects,defective_rate_min,defective,reject,defect_increment,defect,date,module_id,TPPM
0,2024-07-26,F1M7,1,8.28,23,5,5,4,37,60,...,23,3,12,25,4,0,25,2024-07-26,F1M7,4.210606
1,2024-07-26,F1M7,2,8.28,23,5,5,4,37,60,...,9,0,4,28,23,3,31,2024-07-26,F1M7,4.210606
2,2024-07-26,F1M7,3,8.28,23,5,5,4,37,60,...,32,2,13,40,13,3,43,2024-07-26,F1M7,4.210606
3,2024-07-26,F1M7,4,8.28,23,5,5,4,37,60,...,31,4,14,32,1,0,32,2024-07-26,F1M7,4.210606
4,2024-07-26,F1M7,5,8.28,23,5,5,4,37,90,...,27,5,9,30,4,1,31,2024-07-26,F1M7,4.210606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,5,4,29,60,...,12,5,8,27,19,2,29,2024-08-01,F1M8,3.086364
150,2024-08-01,F1M8,8,8.28,14,6,5,4,29,60,...,28,4,14,35,12,2,37,2024-08-01,F1M8,3.086364
151,2024-08-01,F1M8,9,8.28,14,6,5,4,29,60,...,25,2,16,27,3,1,28,2024-08-01,F1M8,3.086364
152,2024-08-01,F1M8,10,8.28,14,6,5,4,29,60,...,23,3,12,25,2,3,28,2024-08-01,F1M8,3.086364


In [24]:
# Now derive defective, reject and defect values
derived_kpis['defective'] = (derived_kpis['target_defective_rate'] * (derived_kpis['ftt'] - derived_kpis['rectified'] + derived_kpis['direct_rejects']))/(100 - 2*derived_kpis['target_defective_rate'])
derived_kpis['defective'] = derived_kpis['defective'].round()
derived_kpis['reject'] = derived_kpis['defective'] - derived_kpis['rectified'] + derived_kpis['direct_rejects']
# derived_kpis['test'] = derived_kpis['defective'] * 100 / (derived_kpis['ftt'] + derived_kpis['defective'] + derived_kpis['reject'])
# derived_kpis['test'] = derived_kpis['test'].round()
derived_kpis['defect_increment'] = np.random.randint(0, 4, size=len(derived_kpis))
derived_kpis['defect'] = derived_kpis['defective'] + derived_kpis['defect_increment']
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,...,rectified,direct_rejects,defective_rate_min,defective,reject,defect_increment,defect,date,module_id,TPPM
0,2024-07-26,F1M7,1,8.28,23,5,5,4,37,60,...,23,3,12,25.0,5.0,3,28.0,2024-07-26,F1M7,4.210606
1,2024-07-26,F1M7,2,8.28,23,5,5,4,37,60,...,9,0,4,28.0,19.0,1,29.0,2024-07-26,F1M7,4.210606
2,2024-07-26,F1M7,3,8.28,23,5,5,4,37,60,...,32,2,13,39.0,9.0,0,39.0,2024-07-26,F1M7,4.210606
3,2024-07-26,F1M7,4,8.28,23,5,5,4,37,60,...,31,4,14,32.0,5.0,3,35.0,2024-07-26,F1M7,4.210606
4,2024-07-26,F1M7,5,8.28,23,5,5,4,37,90,...,27,5,9,30.0,8.0,1,31.0,2024-07-26,F1M7,4.210606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,5,4,29,60,...,12,5,8,27.0,20.0,1,28.0,2024-08-01,F1M8,3.086364
150,2024-08-01,F1M8,8,8.28,14,6,5,4,29,60,...,28,4,14,35.0,11.0,1,36.0,2024-08-01,F1M8,3.086364
151,2024-08-01,F1M8,9,8.28,14,6,5,4,29,60,...,25,2,16,27.0,4.0,3,30.0,2024-08-01,F1M8,3.086364
152,2024-08-01,F1M8,10,8.28,14,6,5,4,29,60,...,23,3,12,25.0,5.0,0,25.0,2024-08-01,F1M8,3.086364


In [25]:
# Edit plan qty and produced_mins
summed_kpis = derived_kpis.groupby(['Date', 'Line']).agg({'ftt': 'sum','rectified': 'sum','reject': 'sum'
}).reset_index()

# Merge the summed data with the plan DataFrame on 'date' and 'module_id'
plan = pd.merge(plan, summed_kpis, left_on=['date', 'module_id'], right_on=['Date', 'Line'] ,how='left')

target_increment = 20
plan['qty'] = plan['ftt'] + plan['rectified'] + plan['reject'] + np.random.randint(0, 20, size=len(plan))
plan['qty'] = plan['qty'].astype(int)
plan['produced_mins'] = plan['qty'] * plan['smv']

plan.drop(columns=['Date', 'Line', 'ftt', 'rectified', 'reject'], inplace=True)
plan['manpower'] = plan['operator'] + plan['helper'] + plan['ironman'] + plan['qc']
plan


,plan_id,plan_type,created_ts,modified_ts,date,shift_id,shift,tenant_id,tenant,cluster_id,...,color,qty,mins,operator,helper,ironman,qc,manpower,smv,produced_mins
0,12f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,2771,660,23,5,5,4,37,8.28,22943.88
1,12f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,2428,660,23,5,5,4,37,9.45,22944.60
2,12f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,2441,660,20,5,5,4,34,8.28,20211.48
3,12f0ffe7-c509-49b5-b203-83be908b2884,2,2024-07-29 10:00:00,2024-07-29 14:00:00,2024-07-29,1,Morning,NaN,NaN,NaN,...,White,2378,660,23,5,5,4,37,10.11,24041.58
4,12f0ffe7-c509-49b5-b203-83be908b2885,2,2024-07-30 10:00:00,2024-07-30 15:00:00,2024-07-30,1,Morning,NaN,NaN,NaN,...,Red,2860,660,23,5,5,4,37,8.21,23480.60
5,12f0ffe7-c509-49b5-b203-83be908b2886,2,2024-07-31 10:00:00,2024-07-31 14:00:00,2024-07-31,1,Morning,NaN,NaN,NaN,...,Black,2409,660,23,5,5,4,37,9.45,22765.05
6,12f0ffe7-c509-49b5-b203-83be908b2887,2,2024-08-01 10:00:00,2024-08-01 13:00:00,2024-08-01,1,Morning,NaN,NaN,NaN,...,Print,3408,660,26,8,5,4,43,8.28,28218.24
7,13f0ffe7-c509-49b5-b203-83be908b2881,2,2024-07-26 10:00:00,2024-07-26 14:00:00,2024-07-26,1,Morning,NaN,NaN,NaN,...,Print,1829,660,12,5,5,4,26,8.28,15144.12
8,13f0ffe7-c509-49b5-b203-83be908b2882,2,2024-07-27 10:00:00,2024-07-27 13:00:00,2024-07-27,1,Morning,NaN,NaN,NaN,...,Black,1692,660,12,5,5,4,26,9.45,15989.40
9,13f0ffe7-c509-49b5-b203-83be908b2883,2,2024-07-28 10:00:00,2024-07-28 14:00:00,2024-07-28,1,Morning,NaN,NaN,NaN,...,Print,1759,660,10,5,5,4,24,8.28,14564.52


In [26]:
# Add TPPM
derived_kpis = derived_kpis.merge(plan[['date', 'module_id', 'qty', 'mins']],
                              left_on=['Date', 'Line'],
                              right_on= ['date','module_id'], how='left')
derived_kpis['TPPM'] = derived_kpis['qty'] / derived_kpis['mins']
derived_kpis.drop(columns=['qty', 'mins'], inplace=True)
derived_kpis

,Date,Line,Hour,smv,Operator,Helper,Iron man,QC,Man power,clock_mins,...,defective_rate_min,defective,reject,defect_increment,defect,date_x,module_id_x,TPPM,date_y,module_id_y
0,2024-07-26,F1M7,1,8.28,23,5,5,4,37,60,...,12,25.0,5.0,3,28.0,2024-07-26,F1M7,4.198485,2024-07-26,F1M7
1,2024-07-26,F1M7,2,8.28,23,5,5,4,37,60,...,4,28.0,19.0,1,29.0,2024-07-26,F1M7,4.198485,2024-07-26,F1M7
2,2024-07-26,F1M7,3,8.28,23,5,5,4,37,60,...,13,39.0,9.0,0,39.0,2024-07-26,F1M7,4.198485,2024-07-26,F1M7
3,2024-07-26,F1M7,4,8.28,23,5,5,4,37,60,...,14,32.0,5.0,3,35.0,2024-07-26,F1M7,4.198485,2024-07-26,F1M7
4,2024-07-26,F1M7,5,8.28,23,5,5,4,37,90,...,9,30.0,8.0,1,31.0,2024-07-26,F1M7,4.198485,2024-07-26,F1M7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2024-08-01,F1M8,7,8.28,14,6,5,4,29,60,...,8,27.0,20.0,1,28.0,2024-08-01,F1M8,3.081818,2024-08-01,F1M8
150,2024-08-01,F1M8,8,8.28,14,6,5,4,29,60,...,14,35.0,11.0,1,36.0,2024-08-01,F1M8,3.081818,2024-08-01,F1M8
151,2024-08-01,F1M8,9,8.28,14,6,5,4,29,60,...,16,27.0,4.0,3,30.0,2024-08-01,F1M8,3.081818,2024-08-01,F1M8
152,2024-08-01,F1M8,10,8.28,14,6,5,4,29,60,...,12,25.0,5.0,0,25.0,2024-08-01,F1M8,3.081818,2024-08-01,F1M8


In [27]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='derived_kpis', index=False)

In [29]:
# Rename to suit the Traget KPI structure

derived_kpis.rename(columns={'smv': 'SMV', 'ftt': 'FTT', 'defective' : 'Defective', 'defect': 'Defect', 'rectified': 'Rectified', 'reject': 'Reject'}, inplace=True)

derived_kpis = derived_kpis[['Date','Line','Hour','FTT','SMV','TPPM','Defective','Defect','Rectified','Reject','Operator','Helper','Iron man','QC','Man power']]


derived_kpis['Defect'] = derived_kpis['Defect'].astype(int)
derived_kpis['Defective'] = derived_kpis['Defective'].astype(int)
derived_kpis['Reject'] = derived_kpis['Reject'].astype(int)

<ipython-input-29-83f607be6bb9>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derived_kpis['Defect'] = derived_kpis['Defect'].astype(int)
<ipython-input-29-83f607be6bb9>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  derived_kpis['Defective'] = derived_kpis['Defective'].astype(int)
<ipython-input-29-83f607be6bb9>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

In [30]:
derived_kpis.head(50)

,Date,Line,Hour,FTT,SMV,TPPM,Defective,Defect,Rectified,Reject,Operator,Helper,Iron man,QC,Man power
0,2024-07-26,F1M7,1,181,8.28,4.198485,25,28,23,5,23,5,5,4,37
1,2024-07-26,F1M7,2,230,8.28,4.198485,28,29,9,19,23,5,5,4,37
2,2024-07-26,F1M7,3,231,8.28,4.198485,39,39,32,9,23,5,5,4,37
3,2024-07-26,F1M7,4,194,8.28,4.198485,32,35,31,5,23,5,5,4,37
4,2024-07-26,F1M7,5,295,8.28,4.198485,30,31,27,8,23,5,5,4,37
5,2024-07-26,F1M7,6,190,8.28,4.198485,50,51,16,37,23,5,5,4,37
6,2024-07-26,F1M7,7,251,8.28,4.198485,20,22,12,13,23,5,5,4,37
7,2024-07-26,F1M7,8,198,8.28,4.198485,38,39,37,1,23,5,4,4,36
8,2024-07-26,F1M7,9,182,8.28,4.198485,31,33,22,9,23,5,5,4,37
9,2024-07-26,F1M7,10,230,8.28,4.198485,26,28,25,5,23,5,5,4,37


In [31]:
# Open the existing Excel file and add the new sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    derived_kpis.to_excel(writer, sheet_name='target_kpis', index=False)

In [32]:
# Update the plan sheet
existing_excel_path = '/content/drive/My Drive/plan_data/constraints.xlsx'

with pd.ExcelWriter(existing_excel_path, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
    plan.to_excel(writer, sheet_name='plan', index=False)